# Tensor Ring Recurrent Neural Network

By replacing input-to-hidden layer of a RNN with tensor ring cores, tensor ring RNN is constructed.

In [1]:
import tednet as tdt
import tednet.tnn.tensor_ring as tr

In [2]:
# Define a TRLSTM
model = tr.TRLSTM([5, 5], [6, 6], [3, 3, 3, 3])

compression_ration is:  10.0


It is easy to define a tensor ring model.
